In [2]:
import string

ascii_characters = string.printable
ascii_characters 

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

In [3]:
ascii_characters = ascii_characters[:-15]
ascii_characters

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@['

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time


driver = webdriver.Chrome()
driver.get('https://onlineasciitools.com/convert-ascii-to-image')

driver.maximize_window()
time.sleep(1)

driver.execute_script('window.scrollTo(0, 900);')

font_dropdown = driver.find_element(By.CSS_SELECTOR, "select.input-option[data-index='typeface']")
font_dropdown.click()

monospace_option = driver.find_element(By.CSS_SELECTOR, "option[value='monospace']")
monospace_option.click()

driver.execute_script('window.scrollTo(0, 100);')   

input_area = driver.find_element(By.CSS_SELECTOR, "textarea.data")
for character in ascii_characters:
    input_area.send_keys(character)
    
    time.sleep(1)
    
    save_btn = driver.find_elements(By.CSS_SELECTOR, "div[data-toggle='toggle-save-as']")[1]
    save_btn.click()
    
    time.sleep(2)
    
    download_btn = driver.find_elements(By.CSS_SELECTOR, "div.widget.widget-save-as")[1]
    download_btn.click()
    
    input_area.clear()

driver.close()

In [4]:
files_names = ['output-onlineasciitools.png'] + [f'output-onlineasciitools ({num}).png' for num in range(1, 85)]
files_directories = [f'ascii_images/{file_name}' for file_name in files_names]

files_directories[:5]

['ascii_images/output-onlineasciitools.png',
 'ascii_images/output-onlineasciitools (1).png',
 'ascii_images/output-onlineasciitools (2).png',
 'ascii_images/output-onlineasciitools (3).png',
 'ascii_images/output-onlineasciitools (4).png']

In [5]:
# because all the files are in the same order as 'ascii_characters' 
# we can just iterate through them and associate them with their respective carachters

character_file_dict = {}

for x in range(85):
    character_file_dict[ascii_characters[x]] = files_directories[x]
    
list(character_file_dict.items())[:5]

[('0', 'ascii_images/output-onlineasciitools.png'),
 ('1', 'ascii_images/output-onlineasciitools (1).png'),
 ('2', 'ascii_images/output-onlineasciitools (2).png'),
 ('3', 'ascii_images/output-onlineasciitools (3).png'),
 ('4', 'ascii_images/output-onlineasciitools (4).png')]

In [6]:
from PIL import Image

img = Image.open('ascii_images/output-onlineasciitools.png').convert('L')

display(img)
print(list(img.getdata())[:10])
print(len(list(img.getdata())))
print(26*48)

[255, 255, 255, 255, 255, 255, 255, 255, 255, 255]
1248
1248


In [7]:
character_brightness_dict = {}

for character in character_file_dict:
    img = Image.open(character_file_dict[character]).convert('L')
    total = sum(list(img.getdata()))
    brightness = total / len(list(img.getdata()))
    
    character_brightness_dict[character] = brightness
    
list(character_brightness_dict.items())[:5]

[('0', 185.8966346153846),
 ('1', 211.54967948717947),
 ('2', 207.18429487179486),
 ('3', 209.43830128205127),
 ('4', 200.50721153846155)]

In [8]:
# now we need to normalize the data

b_max = max(character_brightness_dict.values())
b_min = min(character_brightness_dict.values())

norm_character_brightness_dict = {}

for c in character_brightness_dict:
    norm_character_brightness_dict[c] = round(((character_brightness_dict[c] - b_min) / (b_max - b_min)), 3)
    
list(norm_character_brightness_dict.items())[:5]

[('0', 0.245), ('1', 0.567), ('2', 0.512), ('3', 0.541), ('4', 0.428)]

In [9]:
import pandas as pd

df_data = [{'character': x, 'brightness': norm_character_brightness_dict[x]} for x in norm_character_brightness_dict]
df = pd.DataFrame(df_data)
df.sort_values(by='brightness')

,character,brightness
83,@,0.000
67,&,0.181
16,g,0.183
52,Q,0.196
0,0,0.245
...,...,...
73,",",0.905
77,:,0.920
68,',0.990
74,-,0.992


In [10]:
import json


with open('character_brightness.json', 'w') as file:
    data = json.dumps(norm_character_brightness_dict, indent=4)
    file.write(data)